# Project: Be a hero & save a pet today
## Team Name: In the Dawg Houz
### Team:  Emily Cogsgill, Marya Crigler,  Carlos Pisani, Stephen Schadt

## Dependencies 

In [1]:
# Dependencies
import pandas as pd
import numpy as np
import datetime
import re
import requests
import math

from datetime import datetime
# Google API Key
from config import gkey


## Clean up raw City of Austin Animal Shelter Data
### Intake and Outcome data files downloaded from City of Austin on November 29, 2017

In [2]:
# Set filepaths
csv_filepath1 = "raw data/Austin_Animal_Center_Intakes.csv"
csv_filepath2 = "raw data/Austin_Animal_Center_Outcomes.csv"

In [3]:
#Define reusable functions for cleanse

#Determine purebred status
def GetPurebred(x):
  if "Mix" in x["Breed"] : return 0
  elif "/" in x["Breed"] : return 0
  else: return 1


#Convert the Age to a consistent unit (days)
def GetAgeDays(x):
    if x["AgeUnits"] == "day":
        numDays = x["NumAge"]
    elif x["AgeUnits"] == "week":
        numDays = x["NumAge"] * 7
    elif x["AgeUnits"] == "month":
        numDays = x["NumAge"] * 30
    elif x["AgeUnits"] == "year":
        numDays = x["NumAge"] * 365
    else:
        numDays = 0
    return numDays

#Get intake outcome day length
def get_days_length(val):
    val = str(val)
    days = re.findall('\d*',val)[0]
    return days


### Cleanse Intake file

In [4]:
# Read the csv files into new dataframe
intakes_df = pd.read_csv(csv_filepath1, encoding="iso-8859-1", low_memory=False)
intakes_df.head()

,Animal ID,Name,DateTime,MonthYear,Found Location,Intake Type,Intake Condition,Animal Type,Sex upon Intake,Age upon Intake,Breed,Color
0,A748291,*Madison,05/01/2017 02:26:00 PM,05/01/2017 02:26:00 PM,S Pleasant Valley Rd And E Riverside Dr in Aus...,Stray,Normal,Dog,Intact Female,10 months,Pit Bull Mix,Black
1,A750529,NaN,05/28/2017 01:22:00 PM,05/28/2017 01:22:00 PM,8312 North Ih 35 in Austin (TX),Stray,Normal,Dog,Intact Female,5 months,Miniature Schnauzer Mix,White/Cream
2,A730601,NaN,07/07/2016 12:11:00 PM,07/07/2016 12:11:00 PM,1109 Shady Ln in Austin (TX),Stray,Normal,Cat,Intact Male,7 months,Domestic Shorthair Mix,Blue Tabby
3,A748238,NaN,05/01/2017 10:53:00 AM,05/01/2017 10:53:00 AM,Airport Blvd And Oak Springs Dr in Austin (TX),Stray,Normal,Dog,Intact Male,3 years,Bichon Frise Mix,White
4,A683644,*Zoey,07/13/2014 11:02:00 AM,07/13/2014 11:02:00 AM,Austin (TX),Owner Surrender,Nursing,Dog,Intact Female,4 weeks,Border Collie Mix,Brown/White


In [5]:
# Drop redundant columns
intakes_df.drop(['MonthYear'], axis = 1, inplace = True)
    #intakes_df.head()

In [6]:
# Transform date field and create new Month and Year columns
intakes_df["DateTime"] = pd.to_datetime(intakes_df["DateTime"])
intakes_df["Month"]= intakes_df["DateTime"].dt.month
intakes_df["Year"]= intakes_df["DateTime"].dt.year

In [7]:
# Identify unique values in the Sex upon Intake column
intakes_df["Sex upon Intake"].unique()

array(['Intact Female', 'Intact Male', 'Spayed Female', 'Unknown',
       'Neutered Male', nan], dtype=object)

In [8]:
# Split Sex upon Intake to an IntakeSex and IntakeSpayNeuter columns
intakes_df["Sex"] = intakes_df["Sex upon Intake"].map({"Intact Female":"Female", "Intact Male":"Male", "Spayed Female":"Female", "Neutered Male":"Male","Unknown":"Unknown Sex"})
intakes_df["Female"] = intakes_df["Sex upon Intake"].map({"Intact Female":1, "Intact Male":0, "Spayed Female":1, "Neutered Male":0,"Unknown":0})
intakes_df["Male"] = intakes_df["Sex upon Intake"].map({"Intact Female":0, "Intact Male":1, "Spayed Female":0, "Neutered Male":1,"Unknown":0})
intakes_df["SpayNeuter"] = intakes_df["Sex upon Intake"].map({"Intact Female":"No", "Intact Male":"No", "Spayed Female":"Yes", "Neutered Male":"Yes","Unknown":"No"})
    #intakes_df.head()

In [9]:
# Set Purebred Status
intakes_df['Purebred'] = intakes_df.apply(GetPurebred, axis=1)
    #intakes_df.head()

In [10]:
#Split Age Upon Intake
intakes_df['NumAge'], intakes_df['AgeUnits'] = intakes_df['Age upon Intake'].str.split(' ', 1).str
    #intakes_df.head()

In [11]:
#Make NumAge a numeric colunm for later calclations
intakes_df["NumAge"] = pd.to_numeric(intakes_df["NumAge"])
    #intakes_df.head()

In [12]:
# Identify unique values in the intake age units
intakes_df["AgeUnits"].unique()

array(['months', 'years', 'weeks', 'month', 'year', 'week', 'days', 'day',
       nan], dtype=object)

In [13]:
#Make the intake age unit values consistent
intakes_df["AgeUnits"].replace(["days","weeks","months","years"],["day","week","month","year"], inplace=True)
    #intakes_df["IntakeAgeUnits"].unique()

In [14]:
#Set the Age Days column
intakes_df['AgeDays'] = intakes_df.apply(GetAgeDays, axis=1)
    #intakes_df.head()

In [15]:
# Get metrics on the raw intake file
intakes_df.describe()

,Month,Year,Female,Male,Purebred,NumAge,AgeDays
count,75577.000000,75577.000000,75576.000000,75576.000000,75577.000000,75576.000000,75577.000000
mean,6.689601,2015.309009,0.437388,0.476540,0.067322,3.445300,761.731559
std,3.257848,1.213225,0.496067,0.499453,0.250581,2.908907,1052.134727
min,1.000000,2013.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.000000,2014.000000,0.000000,0.000000,0.000000,1.000000,60.000000
50%,7.000000,2015.000000,0.000000,0.000000,0.000000,2.000000,365.000000
75%,10.000000,2016.000000,1.000000,1.000000,0.000000,5.000000,1095.000000
max,12.000000,2017.000000,1.000000,1.000000,1.000000,25.000000,9125.000000


In [16]:
#Metrics cont'd
intake_recs = len(intakes_df)
intake_dups = intakes_df.duplicated("Animal ID").sum()
intake_types = intakes_df.groupby(["Animal Type"]).size()
intakeOther = intakes_df[intakes_df["Animal Type"]=="Other"]
intakeOtherGrp = intakeOther.groupby(["Breed"], as_index=False)
intakeOtherCnts = intakeOtherGrp["Animal ID"].count()
sort_intakeOtherCnts = intakeOtherCnts.sort_values("Animal ID", ascending=False)

print(f"Number of records in Intakes file: {intake_recs}")
print(f"Duplicate Intake Animal IDs: {intake_dups}")
print(f"Intakes by Animal Type: {intake_types}")
#print(f"Intake of Other group by Animal Type: {sort_intakeOtherCnts}")

Number of records in Intakes file: 75577
Duplicate Intake Animal IDs: 7031
Intakes by Animal Type: Animal Type
Bird           328
Cat          28489
Dog          42590
Livestock        8
Other         4162
dtype: int64


### Clease Outcome file

In [17]:
#  *** Cleanse the Outcomes file  ***
# Read the csv files into new dataframes
outcomes_df = pd.read_csv(csv_filepath2, encoding="iso-8859-1", low_memory=False)
outcomes_df.head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
0,A741715,*Pebbles,01/11/2017 06:17:00 PM,01/11/2017 06:17:00 PM,03/07/2016,Adoption,NaN,Cat,Spayed Female,10 months,Domestic Shorthair Mix,Calico
1,A658751,Benji,11/13/2016 01:38:00 PM,11/13/2016 01:38:00 PM,07/14/2011,Return to Owner,NaN,Dog,Neutered Male,5 years,Border Terrier Mix,Tan
2,A721285,NaN,02/24/2016 02:42:00 PM,02/24/2016 02:42:00 PM,02/24/2014,Euthanasia,Suffering,Other,Unknown,2 years,Raccoon Mix,Black/Gray
3,A746650,Rose,04/07/2017 11:58:00 AM,04/07/2017 11:58:00 AM,04/06/2016,Return to Owner,NaN,Dog,Intact Female,1 year,Labrador Retriever/Jack Russell Terrier,Yellow
4,A750122,Happy Camper,05/24/2017 06:36:00 PM,05/24/2017 06:36:00 PM,04/08/2017,Transfer,Partner,Dog,Intact Male,1 month,Labrador Retriever Mix,Black


In [18]:
# Drop redundant columns
outcomes_df.drop(['MonthYear'], axis = 1, inplace = True)
    #outcomes_df.head()

In [19]:
# Transform date field and create new Month and Year columns
outcomes_df["DateTime"] = pd.to_datetime(outcomes_df["DateTime"])
outcomes_df["Month"]= outcomes_df["DateTime"].dt.month
outcomes_df["Year"]= outcomes_df["DateTime"].dt.year

In [20]:
# Identify unique values in the Sex upon Outcome column
outcomes_df["Sex upon Outcome"].unique()

array(['Spayed Female', 'Neutered Male', 'Unknown', 'Intact Female',
       'Intact Male', nan], dtype=object)

In [21]:
# Split Sex upon Outcome to an OutcomeSex and OutcomeSpayNeuter columns
outcomes_df["Sex"] = outcomes_df["Sex upon Outcome"].map({"Intact Female":"Female", "Intact Male":"Male", "Spayed Female":"Female", "Neutered Male":"Male","Unknown":"Unknown Sex"})
outcomes_df["Female"] = outcomes_df["Sex upon Outcome"].map({"Intact Female":1, "Intact Male":0, "Spayed Female":1, "Neutered Male":0,"Unknown":0})
outcomes_df["Male"] = outcomes_df["Sex upon Outcome"].map({"Intact Female":0, "Intact Male":1, "Spayed Female":0, "Neutered Male":1,"Unknown":0})
outcomes_df["SpayNeuter"] = outcomes_df["Sex upon Outcome"].map({"Intact Female":"No", "Intact Male":"No", "Spayed Female":"Yes", "Neutered Male":"Yes","Unknown":"No"})
    #outcomes_df.head()

In [22]:
# Get purebred 
outcomes_df['Purebred'] = outcomes_df.apply(GetPurebred, axis=1)

In [23]:
# Split out age
outcomes_df['NumAge'], outcomes_df['AgeUnits'] = outcomes_df['Age upon Outcome'].str.split(' ', 1).str

#Make NumAge a numeric colunm for later calclations
outcomes_df["NumAge"] = pd.to_numeric(outcomes_df["NumAge"])

#Make the intake age unit values consistent
outcomes_df["AgeUnits"].replace(["days","weeks","months","years"],["day","week","month","year"], inplace=True)

#Set the Age Days column
outcomes_df['AgeDays'] = outcomes_df.apply(GetAgeDays, axis=1)

In [24]:
# Get metrics on the raw outcomes file
outcomes_df.describe()

,Month,Year,Female,Male,Purebred,NumAge,AgeDays
count,75508.000000,75508.000000,75506.000000,75506.000000,75508.000000,75502.000000,75508.000000
mean,6.799902,2015.293770,0.437767,0.476492,0.067198,3.528211,772.159175
std,3.301991,1.219788,0.496115,0.499450,0.250367,2.901128,1052.357298
min,1.000000,2013.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.000000,2014.000000,0.000000,0.000000,0.000000,2.000000,90.000000
50%,7.000000,2015.000000,0.000000,0.000000,0.000000,2.000000,365.000000
75%,10.000000,2016.000000,1.000000,1.000000,0.000000,5.000000,1095.000000
max,12.000000,2017.000000,1.000000,1.000000,1.000000,25.000000,9125.000000


In [25]:
#Metrics cont'd
outcome_recs = len(outcomes_df)
outcome_dups = outcomes_df.duplicated("Animal ID").sum()
outcome_types = outcomes_df.groupby(["Animal Type"]).size()

print(f"Number of records in Outcomes file: {outcome_recs}")
print(f"Duplicate Outcome Animal IDs: {outcome_dups}")
print(f"Outcomes by Animal Type: {outcome_types}")

Number of records in Outcomes file: 75508
Duplicate Outcome Animal IDs: 6975
Outcomes by Animal Type: Animal Type
Bird           327
Cat          28519
Dog          42498
Livestock        9
Other         4155
dtype: int64


### Merge Intake and Outcome files on animal id

In [26]:
# **** Merge intakes and outcomes files
animalData_df = pd.merge(intakes_df, outcomes_df, on="Animal ID", how="inner", suffixes=('_intake', '_outcome'))
animalData_df.head()

,Animal ID,Name_intake,DateTime_intake,Found Location,Intake Type,Intake Condition,Animal Type_intake,Sex upon Intake,Age upon Intake,Breed_intake,...,Month_outcome,Year_outcome,Sex_outcome,Female_outcome,Male_outcome,SpayNeuter_outcome,Purebred_outcome,NumAge_outcome,AgeUnits_outcome,AgeDays_outcome
0,A748291,*Madison,2017-05-01 14:26:00,S Pleasant Valley Rd And E Riverside Dr in Aus...,Stray,Normal,Dog,Intact Female,10 months,Pit Bull Mix,...,9,2017,Female,1.0,0.0,Yes,0,1.0,year,365.0
1,A750529,NaN,2017-05-28 13:22:00,8312 North Ih 35 in Austin (TX),Stray,Normal,Dog,Intact Female,5 months,Miniature Schnauzer Mix,...,6,2017,Female,1.0,0.0,Yes,0,5.0,month,150.0
2,A730601,NaN,2016-07-07 12:11:00,1109 Shady Ln in Austin (TX),Stray,Normal,Cat,Intact Male,7 months,Domestic Shorthair Mix,...,7,2016,Male,0.0,1.0,Yes,0,7.0,month,210.0
3,A748238,NaN,2017-05-01 10:53:00,Airport Blvd And Oak Springs Dr in Austin (TX),Stray,Normal,Dog,Intact Male,3 years,Bichon Frise Mix,...,5,2017,Male,0.0,1.0,Yes,0,3.0,year,1095.0
4,A683644,*Zoey,2014-07-13 11:02:00,Austin (TX),Owner Surrender,Nursing,Dog,Intact Female,4 weeks,Border Collie Mix,...,11,2014,Female,1.0,0.0,Yes,0,4.0,month,120.0


In [27]:
print(animalData_df.columns)

Index(['Animal ID', 'Name_intake', 'DateTime_intake', 'Found Location',
       'Intake Type', 'Intake Condition', 'Animal Type_intake',
       'Sex upon Intake', 'Age upon Intake', 'Breed_intake', 'Color_intake',
       'Month_intake', 'Year_intake', 'Sex_intake', 'Female_intake',
       'Male_intake', 'SpayNeuter_intake', 'Purebred_intake', 'NumAge_intake',
       'AgeUnits_intake', 'AgeDays_intake', 'Name_outcome', 'DateTime_outcome',
       'Date of Birth', 'Outcome Type', 'Outcome Subtype',
       'Animal Type_outcome', 'Sex upon Outcome', 'Age upon Outcome',
       'Breed_outcome', 'Color_outcome', 'Month_outcome', 'Year_outcome',
       'Sex_outcome', 'Female_outcome', 'Male_outcome', 'SpayNeuter_outcome',
       'Purebred_outcome', 'NumAge_outcome', 'AgeUnits_outcome',
       'AgeDays_outcome'],
      dtype='object')


In [28]:
#Remove rows with duplicate animal IDs
animalData_df=animalData_df.drop_duplicates("Animal ID",keep=False)
len(animalData_df)

62049

In [29]:
#Test to see if any other columns are redundant
animalData_df["Color_intake"].equals(animalData_df["Color_outcome"])

False

In [30]:
#Filter for only Dogs and Cats
animalData_df = animalData_df.loc[animalData_df["Animal Type_intake"].isin(["Dog","Cat"])]
len(animalData_df)

57593

In [31]:
# Calc date diff between intakes and outcomes
animalData_df["Intake Outcome Days"] = animalData_df["DateTime_outcome"]-animalData_df["DateTime_intake"]
#animalData_df.head()

In [32]:
animalData_df["IODays"] = animalData_df['Intake Outcome Days'].apply(get_days_length)
animalData_df["IODays"]=pd.to_numeric(animalData_df["IODays"])
animalData_df.head()

,Animal ID,Name_intake,DateTime_intake,Found Location,Intake Type,Intake Condition,Animal Type_intake,Sex upon Intake,Age upon Intake,Breed_intake,...,Sex_outcome,Female_outcome,Male_outcome,SpayNeuter_outcome,Purebred_outcome,NumAge_outcome,AgeUnits_outcome,AgeDays_outcome,Intake Outcome Days,IODays
0,A748291,*Madison,2017-05-01 14:26:00,S Pleasant Valley Rd And E Riverside Dr in Aus...,Stray,Normal,Dog,Intact Female,10 months,Pit Bull Mix,...,Female,1.0,0.0,Yes,0,1.0,year,365.0,125 days 09:34:00,125.0
1,A750529,NaN,2017-05-28 13:22:00,8312 North Ih 35 in Austin (TX),Stray,Normal,Dog,Intact Female,5 months,Miniature Schnauzer Mix,...,Female,1.0,0.0,Yes,0,5.0,month,150.0,4 days 03:20:00,4.0
2,A730601,NaN,2016-07-07 12:11:00,1109 Shady Ln in Austin (TX),Stray,Normal,Cat,Intact Male,7 months,Domestic Shorthair Mix,...,Male,0.0,1.0,Yes,0,7.0,month,210.0,0 days 20:49:00,0.0
3,A748238,NaN,2017-05-01 10:53:00,Airport Blvd And Oak Springs Dr in Austin (TX),Stray,Normal,Dog,Intact Male,3 years,Bichon Frise Mix,...,Male,0.0,1.0,Yes,0,3.0,year,1095.0,5 days 05:04:00,5.0
4,A683644,*Zoey,2014-07-13 11:02:00,Austin (TX),Owner Surrender,Nursing,Dog,Intact Female,4 weeks,Border Collie Mix,...,Female,1.0,0.0,Yes,0,4.0,month,120.0,115 days 23:04:00,115.0


#### Export merged file to CSV

In [33]:
#Export to CSV
animalData_df.to_csv("raw data/MergedData.csv", encoding="utf-8", index=False)

## Cleanup of Austin vs Louisville Data
### Computing time deltas for Louisville data

In [34]:
louisville_data = pd.read_csv("raw data/Louisville.csv")
louisville_data.head()

,AnimalID,AnimalType,IntakeDate,IntakeType,IntakeSubtype,PrimaryColor,PrimaryBreed,SecondaryBreed,Gender,SecondaryColor,...,IntakeInternalStatus,IntakeAsilomarStatus,ReproductiveStatusAtIntake,OutcomeDate,OutcomeType,OutcomeSubtype,OutcomeReason,OutcomeInternalStatus,OutcomeAsilomarStatus,ReproductiveStatusAtOutcome
0,A366370,CAT,2008-11-07 10:50:00,STRAY,OTC,WHITE,DOMESTIC SHORTHAIR,NaN,NEUTERED MALE,BROWN,...,FEARFUL,HEALTHY,ALTERED,2008-11-12 15:46:00,EUTH,FERAL,NaN,NaN,UNHEALTHY/UNTREATABLE,ALTERED
1,A366531,CAT,2008-11-10 10:20:00,STRAY,OTC,BLACK,DOMESTIC SHORTHAIR,DOMESTIC SHORTHAIR,UNKNOWN,NaN,...,NORMAL,HEALTHY,UNKNOWN,2008-11-19 20:10:00,EUTH,CONTAG DIS,NaN,SICK,HEALTHY,UNKNOWN
2,A532367,BIRD,2014-07-23 23:21:00,CONFISCATE,CRUELTY,RED,CHICKEN,NaN,MALE,BLACK,...,OTHER,HEALTHY,FERTILE,2014-11-05 15:49:00,TRANSFER,NaN,NaN,NaN,HEALTHY,FERTILE
3,A532474,OTHER,2014-07-24 18:29:00,ET REQUEST,NaN,BROWN,BAT,NaN,UNKNOWN,NaN,...,OTHER,HEALTHY,UNKNOWN,2014-07-24 23:59:00,EUTH,MEDICAL,NaN,OTHER,HEALTHY,UNKNOWN
4,A281756,DOG,2006-09-11 18:10:00,OWNER SUR,OTC,WHITE,PIT BULL TERRIER,NaN,MALE,BROWN,...,NORMAL,HEALTHY,FERTILE,2006-09-12 13:44:00,EUTH,TIME/SPACE,NaN,NaN,HEALTHY,FERTILE


In [35]:
louisville_data['Days from Intake to Outcome']=''

for row in range(0,len(louisville_data)):
    #print(row)
    try: 
        intake_date=str(louisville_data.iloc[row,2])
        outcome_date=str(louisville_data.iloc[row,15])                 
        intake_datetime = datetime.strptime(intake_date, '%Y-%m-%d %H:%M:%S')
        outcome_datetime = datetime.strptime(outcome_date, '%Y-%m-%d %H:%M:%S')                 
        days_to_outcome=(outcome_datetime-intake_datetime).total_seconds()/86400
        louisville_data.iloc[row,22]=days_to_outcome
    except ValueError:
        continue

In [36]:
print(louisville_data.columns)

Index(['AnimalID', 'AnimalType', 'IntakeDate', 'IntakeType', 'IntakeSubtype',
       'PrimaryColor', 'PrimaryBreed', 'SecondaryBreed', 'Gender',
       'SecondaryColor', 'DOB', 'IntakeReason', 'IntakeInternalStatus',
       'IntakeAsilomarStatus', 'ReproductiveStatusAtIntake', 'OutcomeDate',
       'OutcomeType', 'OutcomeSubtype', 'OutcomeReason',
       'OutcomeInternalStatus', 'OutcomeAsilomarStatus',
       'ReproductiveStatusAtOutcome', 'Days from Intake to Outcome'],
      dtype='object')


In [37]:
louisville_data.rename(columns={"Time from Intake to Outcome":"Days between Intake and Outcome"})

,AnimalID,AnimalType,IntakeDate,IntakeType,IntakeSubtype,PrimaryColor,PrimaryBreed,SecondaryBreed,Gender,SecondaryColor,...,IntakeAsilomarStatus,ReproductiveStatusAtIntake,OutcomeDate,OutcomeType,OutcomeSubtype,OutcomeReason,OutcomeInternalStatus,OutcomeAsilomarStatus,ReproductiveStatusAtOutcome,Days from Intake to Outcome
0,A366370,CAT,2008-11-07 10:50:00,STRAY,OTC,WHITE,DOMESTIC SHORTHAIR,NaN,NEUTERED MALE,BROWN,...,HEALTHY,ALTERED,2008-11-12 15:46:00,EUTH,FERAL,NaN,NaN,UNHEALTHY/UNTREATABLE,ALTERED,5.20556
1,A366531,CAT,2008-11-10 10:20:00,STRAY,OTC,BLACK,DOMESTIC SHORTHAIR,DOMESTIC SHORTHAIR,UNKNOWN,NaN,...,HEALTHY,UNKNOWN,2008-11-19 20:10:00,EUTH,CONTAG DIS,NaN,SICK,HEALTHY,UNKNOWN,9.40972
2,A532367,BIRD,2014-07-23 23:21:00,CONFISCATE,CRUELTY,RED,CHICKEN,NaN,MALE,BLACK,...,HEALTHY,FERTILE,2014-11-05 15:49:00,TRANSFER,NaN,NaN,NaN,HEALTHY,FERTILE,104.686
3,A532474,OTHER,2014-07-24 18:29:00,ET REQUEST,NaN,BROWN,BAT,NaN,UNKNOWN,NaN,...,HEALTHY,UNKNOWN,2014-07-24 23:59:00,EUTH,MEDICAL,NaN,OTHER,HEALTHY,UNKNOWN,0.229167
4,A281756,DOG,2006-09-11 18:10:00,OWNER SUR,OTC,WHITE,PIT BULL TERRIER,NaN,MALE,BROWN,...,HEALTHY,FERTILE,2006-09-12 13:44:00,EUTH,TIME/SPACE,NaN,NaN,HEALTHY,FERTILE,0.815278
5,A451184,BIRD,2012-01-29 15:25:00,OWNER SUR,FIELD,BLACK,CHICKEN,NaN,UNKNOWN,WHITE,...,HEALTHY,UNKNOWN,2012-02-22 23:59:00,TRANSFER,RESCUE GRP,NaN,NORMAL,HEALTHY,UNKNOWN,24.3569
6,A278854,CAT,2006-08-17 14:28:00,STRAY,OTC,BROWN TIGER,DOMESTIC SHORTHAIR,NaN,FEMALE,NaN,...,HEALTHY,FERTILE,2006-08-28 11:30:00,EUTH,MEDICAL,NaN,NaN,HEALTHY,FERTILE,10.8764
7,A278862,CAT,2006-08-17 14:28:00,STRAY,OTC,GRAY,DOMESTIC SHORTHAIR,NaN,MALE,NaN,...,HEALTHY,FERTILE,2006-08-28 09:20:00,DIED,IN KENNEL,NaN,NaN,HEALTHY,FERTILE,10.7861
8,A278889,CAT,2006-08-17 16:53:00,STRAY,FIELD,WHITE,DOMESTIC SHORTHAIR,NaN,FEMALE,GRAY,...,HEALTHY,FERTILE,2006-08-22 11:15:00,EUTH,MEDICAL,NaN,NaN,HEALTHY,FERTILE,4.76528
9,A279309,CAT,2006-08-22 18:36:00,STRAY,FIELD,BLACK,DOMESTIC SHORTHAIR,NaN,FEMALE,NaN,...,HEALTHY,FERTILE,2006-09-07 15:34:00,DIED,IN KENNEL,NaN,NaN,HEALTHY,FERTILE,15.8736


In [38]:
louisville_data=louisville_data.drop(columns=['Days from Intake to Outcome'])

TypeError: drop() got an unexpected keyword argument 'columns'

In [ ]:
louisville_data=louisville_data.rename(columns={"Time from Intake to Outcome":"Days between Intake and Outcome"})
louisville_data.head()

In [ ]:
louisville_data.to_csv('raw data/test/Louisville_with_time_deltas.csv')

In [ ]:
louisville_data_dogs_cats_only=louisville_data[(louisville_data['AnimalType']=='CAT') | (louisville_data['AnimalType']=='DOG')]

In [ ]:
#confirm it worked
louisville_data_dogs_cats_only['AnimalType'].value_counts()

In [ ]:
louisville_data_dogs_cats_only.to_csv('raw data/test/Louisville_with_time_deltas_dogs_cats_only.csv')

### Computing time deltas for Austin data

In [ ]:
#Read in the data
austin_data = pd.read_csv("raw data/MergedData.csv")

In [ ]:
austin_data['Days from Intake to Outcome']=''

for row in range(len(austin_data)):
    #print(f"Processing row {row}")
    try: 
        intake_date=str(austin_data.loc[row,'DateTime_intake'])
        outcome_date=str(austin_data.loc[row,'DateTime_outcome'])    
        intake_datetime = datetime.strptime(intake_date, '%m/%d/%Y %H:%M:%S %p')
        outcome_datetime = datetime.strptime(outcome_date, '%m/%d/%Y %H:%M:%S %p')                 
        days_to_outcome=(outcome_datetime-intake_datetime).total_seconds()/86400
        austin_data.loc[row,'Days from Intake to Outcome']=days_to_outcome
    except ValueError:
        continue

In [ ]:
#confirm there are only cats and dogs
austin_data['Animal Type_intake'].value_counts()

In [ ]:
austin_data.to_csv('raw data/test/Austin_merged_data_with_time_deltas.csv')

## Animal Shelter Intake Analysis in Austin, TX by Location

#### Build animal / location dataframes

In [ ]:
#
# Filtered Dataframe: Only intakes from 1/1/16, 12/1/16, 6/1/17, or 9/1/17 and forward 
#  (depending on how many API calls we can make)
#

# *** Below datasets contain larger datasets and should only be loaded if you have ample API calls to make against Google Maps API ***
# df_intakes_2016_and_on = pd.read_csv('raw data/Austin_Animal_Center_Intakes_2016_and_on.csv', encoding='latin-1')
# df_intakes_2017_and_on = pd.read_csv('raw data/Austin_Animal_Center_Intakes_2017_and_on.csv', encoding='latin-1')
# df_intakes_2017_and_on = pd.read_csv('raw data/Austin_Animal_Center_Intakes_092017_and_on.csv', encoding='latin-1')

# *** Below dataset only contains 1000ish records - use this one for testing purposes ***
df_intakes_2017_and_on = pd.read_csv('raw data/Austin_Animal_Center_Intakes_2017-11_and_on.csv', encoding='latin-1')

# Create clean dataframe to populate only rows with applicable addresses
df_intakes_clean = pd.DataFrame(columns=["DateTime", "Found Address", "Intake Type", "Intake Condition",
                                        "Animal Type", "Sex upon Intake", "Age upon Intake", "Breed", "Color"])

In [ ]:
print(f"Length of dataset: {len(df_intakes_2017_and_on)}")
df_intakes_2017_and_on.head()

In [ ]:
# *********************************************
# *** Function to clean address column data ***
# *********************************************
def clean_address(addr):
    '''
    Function: clean_address
    Argument: address
    Return values: tuple final address (string), is_full_address (boolean)
    '''
    # variable determining whether or not this is a full address (defaults to False)
    is_full_address = False

    # Initialize address variable by cleaning off the (TX) part
    addressclean = addr.replace(" (TX)","")

    # Split the address from the city
    address = addressclean.split(" in ")

    # Clean up address
    address_words = addressclean.split(" ")

    # First find out if this address is not applicable
    if addressclean == "Outside Jurisdiction":
        address_final = "NA"
    
    # Next, find out if this is an actual street address
    elif (len(address) > 1):
        is_full_address = True

        # Street address (raw)
        address_street = address[0]
        address_city = address[1]
        
        # Clean up addresses with "/" characters into [street1 and street2] syntax
        address_corner = address_street.split("/")
        if len(address_corner) > 1:
            address_street = f"{address_corner[0]} and {address_corner[1]}"
        else:
            address_street = address_corner[0]
            
        address_final = f"{address_street},{address_city},TX"
        
    # Finally, for non-address strings...single-city listing
    else:
        address_final = f"{addressclean},TX"
        
    return (address_final, is_full_address)

In [ ]:
#
# Loop through last-1-year or last-2-years dataset, and only insert rows with clean addresses into clean dataset
#
for index, row in df_intakes_2017_and_on.iterrows():
    # Call function to clean up address into something we can pass to Google API
    address_tuple = clean_address(row["Found Location"])
    address = address_tuple[0]
    is_full_address = address_tuple[1]
    
    if address == "NA":
        print("Outside jurisdiction - skipping")
        continue
    elif is_full_address == False:
        print("No actual address - skipping")
        continue
    else:
        # Fill empty "clean" dataframe with rows we actually want to process
        df_intakes_clean = df_intakes_clean.append({"DateTime": row["DateTime"],
                                "Found Address": address,
                                "Intake Type": row["Intake Type"],
                                "Intake Condition": row["Intake Condition"],        
                                "Animal Type": row["Animal Type"],
                                "Sex upon Intake": row["Sex upon Intake"],
                                "Age upon Intake": row["Age upon Intake"],
                                "Breed": row["Breed"],
                                "Color": row["Color"]}, ignore_index=True)
        

In [ ]:
# Visualize cleaned dataset
print(f"Length of entire dataset: {len(df_intakes_clean)}")
df_intakes_clean["Intake Type"].value_counts()
df_intakes_clean.head()

### Google Maps API integration

In [ ]:
# Create summary dataframe to house address, lat, long, and boolean indicating if this pet lived in a home
df_summary = pd.DataFrame(columns=["Address", "Latitude", "Longitude", "Pet at Home", "Animal Type"])

#### Function and For-loop to grab lat/lng from Google API

In [ ]:
# If we have already populated the summary dataframe, drop any rows where Lat/Long values are zero, 
#  so appending below will be clean
df_summary = df_summary[df_summary.Latitude != 0]

# Visualize the trimmed dataframe
print(f"Length of dataset: {len(df_summary)}")
df_summary.head()

In [ ]:
# **************************************************************
# *** Function to pull lat / lng values from Google Maps API ***
# **************************************************************
def get_lat_long(address):
    '''
    Function: get_lat_long
    Purpose:  Get lat and long codes from Google maps API, given an address string
    Argument: address
    Returns:  lat, long values
    '''
    # Create endpoint URL
    endpoint_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={gkey}"

    # Run request to grab the JSON at the requested URL
    google_api_json = requests.get(endpoint_url).json()

    # Append the lat/lng to the appropriate columns (use try / except to skip addresses with errors)
    try: 
        lat = google_api_json["results"][0]["geometry"]["location"]["lat"]
        lng = google_api_json["results"][0]["geometry"]["location"]["lng"]
        retval = (lat, lng)

    except IndexError:
        retval = (0, 0)

    return retval
# Initialize loop variables
row_count = 0 
processed_addresses = []

#
# Loop through cleaned dataset and determine lat/lng using Google maps geocoding API
#
# ** NOTE: this loop must not exceed 25,000 calls to the Google API in a 24 hour period, per the Google free API terms ***
#
for index, row in df_intakes_clean.iterrows():
    row_count += 1
    
    # Set address and pet_at_home boolean variable
    addr = row["Found Address"]
    pet_at_home = False
    
    # If we've already done a lookup for this address, no need to call Maps API
    if (addr in processed_addresses):
        print(f"Address already processed: {addr}")
        continue
    else:
        # Verify if this address already has a lat/long value in the table. If so, continue. 
        is_address_in_df = df_summary[df_summary.Address == addr].count()["Address"]
       
        # If this address isn't already in the dataframe, call Google API to populate lat/lon
        if (is_address_in_df == 0):
            print(f"New address being processed: {addr}: {str(row_count)}")
            (latitude, longitude) = get_lat_long(addr)
            
            # Append to addresses array to mark this address as processed
            processed_addresses.append(addr)
        else:
            # Address was found, but the Latitude value is populated
            print(f"Address already populated: {addr}")
            continue
       
    # Set variables for "Pet at Home", "Animal Type"
    animal_type = row["Animal Type"]
    if row["Intake Type"] == "Owner Surrender" or row["Intake Type"] == "Euthanasia Request" or row["Intake Type"] == "Public Assist":
        pet_at_home = True

    # Append values to our summary dataframe
    df_summary = df_summary.append({"Address": addr,
                                    "Latitude": latitude,
                                    "Longitude": longitude,
                                    "Pet at Home": pet_at_home,
                                    "Animal Type": animal_type},
                                    ignore_index=True)

In [ ]:
print(f"Length of summary dataset: {len(df_summary)}")
#df_summary.head()

# Save the DataFrame as a csv
df_summary.to_csv("animal_shelter_analysis_summary_clean_LocationData.csv", encoding="utf-8", index=False)

### Create summary dataframes: 2016 to Present and Animals from Homes
#### Create filtered datasets (animals in homes, cats, dogs)

In [ ]:
# *** Filter original dataframe into smaller datasets ***

# All intakes not 'Wildlife' or 'Stray' Intake Type: includes "Intake Type" of:
#     - Euthanasia request
#     - Owner surrender
#     - Public assist
df_animals_homes = df_intakes_clean.loc[((df_intakes_clean["Intake Type"] =="Owner Surrender") | 
                                         (df_intakes_clean["Intake Type"] == "Euthanasia Request") |
                                         (df_intakes_clean["Intake Type"] == "Public Assist")),]

# Dogs only
df_animals_dogs = df_intakes_clean.loc[(df_intakes_clean["Animal Type"] =="Dog"),]

# Cats only 
df_animals_cats = df_intakes_clean.loc[(df_intakes_clean["Animal Type"] =="Cat"),]

# Visualize homes dataset
df_animals_homes.head()

In [ ]:
df_animals_dogs.head()

In [ ]:
# Strays
df_animals_strays = df_intakes_clean.loc[(df_intakes_clean["Intake Type"] =="Stray"),]
print(f"Length of strays dataset: {len(df_animals_strays)}")

#### Create filtered dataframes of unique address lists

In [ ]:
# Calculate address counts for master dataset
address_counts_all = df_intakes_clean["Found Address"].value_counts()
df_address_counts_all = pd.Series.to_frame(address_counts_all).reset_index()
df_address_counts_all = df_address_counts_all.rename(columns={'index': 'Address', 'Found Address': 'Count'})
df_address_counts_all.head()

In [ ]:
# Function to create cleaned dataframe each filtered addresses dataset
def convert_address_counts_to_df(address_counts):
    '''
    Function: convert_address_counts_to_df
    Description: Convert address value counts to dataframe
    Arguments: Series
    Returns:  Dataframe
    '''
    df_address_counts = pd.Series.to_frame(address_counts).reset_index()
    df_address_counts = df_address_counts.rename(columns={'index': 'Address', 'Found Address': 'Count'})
    return df_address_counts

# Create pets-in-homes-specific counts dataframe
address_counts_homes = df_animals_homes["Found Address"].value_counts()
df_address_counts_homes = convert_address_counts_to_df(address_counts_homes)

# Create dogs-specific counts dataframe
address_counts_dogs = df_animals_dogs["Found Address"].value_counts()
df_address_counts_dogs = convert_address_counts_to_df(address_counts_dogs)

# Create cats-specific counts dataframe 
address_counts_cats = df_animals_cats["Found Address"].value_counts()
df_address_counts_cats = convert_address_counts_to_df(address_counts_cats)

# Create strays-specific counts dataframe 
address_counts_strays = df_animals_strays["Found Address"].value_counts()
df_address_counts_strays = convert_address_counts_to_df(address_counts_strays)

# Visualize Strays address dataframe
df_address_counts_strays.head()

In [ ]:
# Visualize pets-in-homes addresses counts
df_address_counts_homes.head()

#### Create filtered summary dataframes with only valid Latitude/Longitude values

In [ ]:
# All animals
df_animals_summary_all = df_summary.loc[(df_summary["Latitude"] != 0),]

# All animals in homes
df_animals_summary_homes = df_summary.loc[(df_summary["Pet at Home"] == True) &
                                                         (df_summary["Latitude"] != 0),]

# Dogs only
df_animals_summary_dogs = df_summary.loc[(df_summary["Animal Type"] == "Dog") &
                                                         (df_summary["Latitude"] != 0),]

# Cats only 
df_animals_summary_cats = df_summary.loc[(df_summary["Animal Type"] == "Cat") & 
                                                         (df_summary["Latitude"] != 0),]

In [ ]:
# Visualize filtered dataframes
print(f"Length of 'All Animals' summary dataframe: {len(df_animals_summary_all)}")
print(f"Length of 'Animals in Homes' summary dataframe: {len(df_animals_summary_homes)}")
df_animals_summary_all.head()

In [ ]:
df_address_counts_cats.head()

#### Merge intake address counts into summary dataframes

In [ ]:
# Merge address count into master summary dataset
df_summary_all = pd.merge(df_animals_summary_all, df_address_counts_all, on="Address")

In [ ]:
# Export and visualize summary of all address counts
df_summary_all.to_csv('raw data/animal_shelter_analysis_address_counts_ALL.csv', encoding='latin-1', index=False)
df_summary_all.head()

In [ ]:
# Merge address count into pets-in-homes dataset
df_summary_homes = pd.merge(df_animals_summary_homes, df_address_counts_homes, on="Address")

In [ ]:
# Export and visualize summary of all address counts
df_summary_homes.to_csv('raw data/animal_shelter_analysis_address_counts_HOMES.csv', encoding='latin-1', index=False)
df_summary_homes.head()

In [ ]:
# Merge address count into dogs & cats dataset
df_summary_dogs = pd.merge(df_animals_summary_dogs, df_address_counts_dogs, on="Address")
df_summary_cats = pd.merge(df_animals_summary_cats, df_address_counts_cats, on="Address")

In [ ]:
# Export and visualize summary of Dogs address counts
df_summary_dogs.to_csv('raw data/animal_shelter_analysis_address_counts_DOGS.csv', encoding='latin-1', index=False)
df_summary_dogs.head()

In [ ]:
# Export and visualize summary of Cats address counts
df_summary_cats.to_csv('raw data/animal_shelter_analysis_address_counts_CATS.csv', encoding='latin-1', index=False)
df_summary_cats.head()

In [ ]:
# Merge address count into strays dataset
df_summary_strays = pd.merge(df_summary, df_address_counts_strays, on="Address")

In [ ]:
# Export and visualize summary of Dogs address counts
df_summary_strays.to_csv('raw data/animal_shelter_analysis_address_counts_STRAYS.csv', encoding='latin-1', index=False)
df_summary_strays.head()

### Dataframes for plotting number of veterinarians vs. number of pet intakes

In [ ]:
# Add necessary column to plotting datasets
df_animals_summary_all["Vet Count"] = ""
df_animals_summary_homes["Vet Count"] = ""
df_animals_summary_dogs["Vet Count"] = ""
df_animals_summary_cats["Vet Count"] = ""

#### Loop through dataset and assign Vet count values by calling Google Radarsearch API

In [ ]:
df_animals_summary_all.head()

In [ ]:
# Counter
row_count = 0

# Loop through and run Google search to get all banks in 5 mile radius (8000 meters)
for index, row in df_animals_summary_all.iterrows():
    
    # Create endpoint url using Google Places Radar and the lat/lng we identified earlier
    #  - Radius search of roughly 1 mile
    #  - Places type "veterinary_care" only
    target_url =f"https://maps.googleapis.com/maps/api/place/radarsearch/json?location={row['Latitude']},{row['Longitude']}&radius=1700&type=veterinary_care&key={gkey}"

    # This link helps to handily see the JSON generated for each query
    print(f"Now retrieving address #{row_count}: {df_animals_summary_all.loc[index]['Address']}")
    row_count += 1 
    
    # Run request to retrieve JSON from target URL (only if it hasn't been set yet)
    if df_animals_summary_all.loc[index]['Vet Count'] == 0 or df_animals_summary_all.loc[index]['Vet Count'] == "":
        vet_data = requests.get(target_url).json()
        
        # Retrieve vet count via number of results within the radius (2500 meters)
        vet_count = len(vet_data["results"])  
        print(f"Final Vet Count for address '{row['Address']}': {str(vet_count)}")
        print("")    
    
        # Store the vet count into the Data Frame
        df_animals_summary_all.set_value(index, "Vet Count", vet_count)
    else:
        print(f"Vet Count already set for this address: {row['Address']}")
        
    # Reset vet_count, so a previous record cannot influence a later one
    vet_count = 0

# Visualize the new dataset
df_animals_summary_all.head()

In [ ]:
# Save the dataframe to CSV
df_animals_summary_all.to_csv('raw data/animal_shelter_analysis_with_Vet_data.csv', encoding='latin-1', index=False)

#### Add number of intake addresses within each vet's lat/long combination to dataframe

In [ ]:
import math

In [ ]:
# Add new tracking column for number of intakes within radius
df_animals_summary_all["Intakes within Radius"] = ""

# Function to calculate if a given lat/long point is contained in the Google place's kilometer radius
def is_location_within_1700meters(check_point_lat, check_point_long, center_point_lat, center_point_long, radius_km):
    '''
    Function: is_location_within_1700meters
    Purpose:  Given lat long values for center point and check point, figure out whether or not check points are within
                X kilometers (in our case, 1.7)
    '''
    
    km_lat = 40000 / 360
    km_lng = math.cos(math.pi * center_point_lat/180) * km_lat
    dst_x = math.fabs(center_point_long - check_point_long) * km_lng
    dst_y = math.fabs(center_point_lat - check_point_lat) * km_lat
    
    return math.sqrt(dst_x * dst_x + dst_y * dst_y) <= radius_km;

In [ ]:
# Radius should be 1.7km, to match 1700 meter Google Radarsearch lookup radius
radius = 1.7 

# Loop through dataset and assign how many intake locations are within given lat/long combination
row_count = 0
for index, row in df_animals_summary_all.iterrows():
    # This link helps to handily see the JSON generated for each query
    print(f"Now retrieving address #{row_count}: {df_animals_summary_all.loc[index]['Address']}")
    row_count += 1 

    # Reset inner loop count / boolean variables
    is_found = False
    num_found = 0
    
    # Loop through dataframe again, and determine how many lat/long combinations are within the current lat/long's 1700 meter radius
    for i_inner, r_inner in df_animals_summary_all.iterrows():    
        is_found = is_location_within_1700meters(r_inner["Latitude"], r_inner["Longitude"], row["Latitude"], row["Longitude"], radius)
        if is_found == True:
            num_found += 1
            
    # Set the number of found intakes for this center point 
    df_animals_summary_all.set_value(index, "Intakes within Radius", num_found)

In [ ]:
# Save the dataframe to CSV, and visualize it
df_animals_summary_all.to_csv('raw data/animal_shelter_analysis_with_VetAndRadius_data.csv', encoding='latin-1', index=False)
df_animals_summary_all.head()